Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [1]:
import retinoto_py as fovea
args = fovea.Params()
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurentperrinet	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=True, num_epochs=10, n_train_stop=0, seed=1998, batch_size=64, num_workers=4, model_name='resnet18', do_scratch=False, lr=0.02, delta1=0.1, delta2=0.001, weight_decay=0.001, shuffle=True, verbose=True)

# testing a ResNet50 model on the validation dataset

In [2]:
VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'val'

In [3]:
val_loader, class_to_idx, idx_to_class = fovea.get_loader(args, VAL_DATA_DIR)

In [4]:
model = fovea.load_model(args)
model.eval()
# --- Parameter Counts ---
param_stats = fovea.count_parameters(model)
print("🔢 Parameter Count:")
print(f"  Total:     {param_stats['total_parameters']:,}")
print(f"  Trainable: {param_stats['trainable_parameters']:,}")
print("-" * 50)

# --- Layer Counts ---
print("🧱 Layer Count:")
# 1. Total number of nn.Module objects
total_layers = fovea.count_layers(model)
print(f"  Total Modules (nn.Module): {total_layers}")

# 2. Counting specific layer types
from torch import nn

conv_layers = fovea.count_layers(model, layer_type=nn.Conv2d)
linear_layers = fovea.count_layers(model, layer_type=nn.Linear)
print(f"  Convolutional (nn.Conv2d): {conv_layers}")
print(f"  Linear (nn.Linear): {linear_layers}")
print("=" * 50)

🔢 Parameter Count:
  Total:     11,689,512
  Trainable: 11,689,512
--------------------------------------------------
🧱 Layer Count:
  Total Modules (nn.Module): 68
  Convolutional (nn.Conv2d): 20
  Linear (nn.Linear): 1


In [5]:
json_filename = args.data_cache / '11_model_accuracy.json'

# %rm -f {json_filename} # Remove previous results to force re-evaluation

if json_filename.exists():
    results = fovea.pd.read_json(json_filename)
else:
    all_results_list = []
    for dataset in fovea.all_datasets:
        VAL_DATA_DIR = args.DATAROOT / f'Imagenet_{dataset}' / 'val'
        val_loader, class_to_idx, idx_to_class = fovea.get_loader(args, VAL_DATA_DIR)
        accuracy = fovea.get_validation_accuracy(args, model, val_loader, f"Evaluating {args.model_name} on dataset: {dataset}")
        all_results_list.append({'model_name':args.model_name, 'dataset':dataset, 'accuracy': accuracy})

    results = fovea.pd.DataFrame(all_results_list)
    results['accuracy_str'] = results['accuracy'].apply(lambda x: f"Accuracy: {x * 100:.1f}%")
    results.to_json(json_filename, orient='records', indent=2)

print(f"Evaluation complete.")

Evaluating resnet18 on dataset: full:   0%|          | 0/782 [00:00<?, ?it/s]

Evaluating resnet18 on dataset: bbox:   0%|          | 0/1258 [00:00<?, ?it/s]

Evaluation complete.


In [6]:
results

,model_name,dataset,accuracy,accuracy_str
0,resnet18,full,0.47854,Accuracy: 47.9%
1,resnet18,bbox,0.46576,Accuracy: 46.6%


In [7]:
for _, row in results.iterrows():
    accuracy_percent = row['accuracy'] * 100
    print(f"Accuracy for {row['dataset']}: {accuracy_percent:.3f}%")

Accuracy for full: 47.854%
Accuracy for bbox: 46.576%


In [8]:
%cat {json_filename}

[
  {
    "model_name":"resnet18",
    "dataset":"full",
    "accuracy":0.47854,
    "accuracy_str":"Accuracy: 47.9%"
  },
  {
    "model_name":"resnet18",
    "dataset":"bbox",
    "accuracy":0.4657604036,
    "accuracy_str":"Accuracy: 46.6%"
  }
]

In [9]:
print(results[['dataset', 'accuracy_str']].to_string(index=False))


dataset    accuracy_str
   full Accuracy: 47.9%
   bbox Accuracy: 46.6%
